In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
import os
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
import peakutils
from matplotlib import colors, cm
from scipy.signal import decimate
import copy
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
from utils.plotting import HeatMapParams
from utils.plotting import heat_map_and_mean
from utils.plotting import get_photometry_around_event
from scipy import stats
from utils.value_change_utils import open_experiment, CustomAlignedDataRewardBlocks, get_all_experimental_records, get_block_change_info, add_traces_and_peaks, plot_mean_trace_for_condition, one_session_get_block_changes

In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
root_directory = 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis\\utils\\'
#figure_dir = 'W:\\photometry_2AC\\plots\\SNL_photo21_26\\value_change_experiment'
figure_dir = 'W:\\photometry_2AC\\plots\\SNL_photo28_35\\value_change_experiment'
processed_data_dir = os.path.join(root_directory, 'value_change_data')
block_data_file = os.path.join(processed_data_dir, 'block_data_nacc_mice.csv')
all_reward_block_data = pd.read_pickle(block_data_file)

In [6]:
all_reward_block_data

,block number,trial number,peak size,relative reward amount,session,mouse,contra reward amount,traces,time points
0,1.0,2.0,3.413520,-4.0,20201218,SNL_photo28,2.0,"[2.2181186254609213, 2.217646941926041, 2.2171...","[-8.0, -7.999899999374996, -7.999799998749992,..."
1,1.0,3.0,0.247170,-4.0,20201218,SNL_photo28,2.0,"[0.8192272556469181, 0.8191840278242005, 0.819...","[-8.0, -7.999899999374996, -7.999799998749992,..."
2,1.0,4.0,0.599085,-4.0,20201218,SNL_photo28,2.0,"[-1.4097600488294875, -1.4099399680014688, -1....","[-8.0, -7.999899999374996, -7.999799998749992,..."
3,1.0,5.0,2.727565,-4.0,20201218,SNL_photo28,2.0,"[1.8196138715412018, 1.8195591109368123, 1.819...","[-8.0, -7.999899999374996, -7.999799998749992,..."
4,1.0,6.0,2.433758,-4.0,20201218,SNL_photo28,2.0,"[-0.3130371907154529, -0.31281892105840314, -0...","[-8.0, -7.999899999374996, -7.999799998749992,..."
...,...,...,...,...,...,...,...,...,...
571,5.0,290.0,1.129411,4.0,20201218,SNL_photo31,6.0,"[0.8058008521634317, 0.8065799400496406, 0.807...","[-8.0, -7.999899999374996, -7.999799998749992,..."
572,5.0,294.0,2.172790,4.0,20201218,SNL_photo31,6.0,"[-1.0082814527300497, -1.0080715658442887, -1....","[-8.0, -7.999899999374996, -7.999799998749992,..."
573,5.0,295.0,0.440540,4.0,20201218,SNL_photo31,6.0,"[0.6246871072985419, 0.6244691700782977, 0.624...","[-8.0, -7.999899999374996, -7.999799998749992,..."
574,5.0,296.0,2.607107,4.0,20201218,SNL_photo31,6.0,"[-0.6303309960346045, -0.6307100415940298, -0....","[-8.0, -7.999899999374996, -7.999799998749992,..."


In [17]:
timepoints = all_reward_block_data['time points'].iloc[0]
font = {'size'   : 12}

matplotlib.rc('font', **font)

mouse_name = 'SNL_photo32'

fig, ax = plt.subplots(2,1) #, figsize=(10,16))
plot_mean_trace_for_condition(ax[0], all_reward_block_data[all_reward_block_data['mouse'] == mouse_name], timepoints,
                              'contra reward amount', error_bar_method='ci', save_location=processed_data_dir)
lg1 = ax[0].legend(title='Absolute value (ul)',bbox_to_anchor=(1., .8, .15,.2), loc='upper left')
plot_mean_trace_for_condition(ax[1], all_reward_block_data[all_reward_block_data['mouse'] == mouse_name], timepoints,
                              'relative reward amount', error_bar_method = 'ci', save_location=processed_data_dir)
lg2 = ax[1].legend(title='Relative value (ul)',bbox_to_anchor=(1., .8, .15,.2), loc='upper left')

plt.tight_layout()


plt.savefig(os.path.join(figure_dir, 'example_mouse{}.pdf'.format(mouse_name)))


Bootstrapping...: 100%|████████████████████| 1000/1000 [00:16<00:00, 61.50it/s]


In [10]:
df1 = all_reward_block_data[['mouse','trial number','session','contra reward amount', 'peak size']]
df1 = df1.pivot_table(index=['mouse', 'contra reward amount'], values=['peak size'], aggfunc=np.nanmean)
df1 = df1.reset_index()
slope, intercept, r_val, p_val1, std_error = stats.linregress(df1['contra reward amount'], 
                                                             df1['peak size'])

In [11]:
df2 = all_reward_block_data[['mouse','trial number','session','relative reward amount', 'peak size']]
df2 = df2.pivot_table(index=['mouse', 'relative reward amount'], values=['peak size'], aggfunc=np.nanmean)
df2 = df2.reset_index()
slope, intercept, r_val, p_val2, std_error = stats.linregress(df2['relative reward amount'], 
                                                             df2['peak size'])


In [12]:
font = {'size'   : 14}
matplotlib.rc('font', **font)

fig, axs = plt.subplots(2,1)
condition1 = 'contra reward amount'
sns.lineplot(ax=axs[0], x=condition1, y='peak size', data=df1, err_style='bars')
axs[0].text(4.5, 1.195, 'p-value = {0:.3f}'.format(p_val1))

condition2 = 'relative reward amount'
sns.lineplot(ax=axs[1], x=condition2, y='peak size', data=df2, err_style='bars')
axs[1].text(1.5, 1.3,'p-value = {0:.3f}'.format(p_val2))

for ax in axs:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_ylabel('response (z-score)')
axs[0].set_xlabel('absolute value (ul)')
axs[1].set_xlabel('relative value (ul)')
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, 'group_data_peaks_vs_value.pdf'))

In [13]:
plt.tight_layout()

In [14]:
colours = cm.inferno(np.linspace(0, 1, 5))
fig, ax = plt.subplots(1,1)
for trial_type in range(1,6):
    try:
        one_side_data = all_reward_block_data[trial_type-1].contra_data
        mean_trace = decimate(one_side_data.mean_trace, 10)
        time_points = decimate(one_side_data.time_points, 10)
        traces = decimate(one_side_data.sorted_traces, 10)
        ax.plot(time_points, mean_trace, lw=1.5, color=colours[trial_type-1], label= trial_type)
        ax.axvline(0, color='k')
    except:
        pass
plt.legend()


No handles with labels found to put in legend.


In [3]:
block_types = pd.DataFrame({'block type': [1, 2, 3, 4, 5], 'left reward': [6, 4, 2, 2, 2], 'right reward': [2, 2, 2, 4, 6]})

In [103]:
mouse_id = 'SNL_photo21'
sessions = ['20200917', '20200918', '20200921']
for session_num, date in enumerate(sessions):
    block_change_info, timepoints = one_session_get_block_changes(mouse_id, date, block_types)
    if session_num == 0:
        all_block_changes = block_change_info
    else:
         all_block_changes = pd.concat([all_block_changes,block_change_info], ignore_index=True)

(106, 160000)
(186, 160000)
(220, 160000)


In [106]:
plot_mean_trace_for_condition(all_block_changes, timepoints, 'new reward amounts')
plot_mean_trace_for_condition(all_block_changes, timepoints, 'change in reward amounts')
plot_mean_trace_for_condition(all_block_changes, timepoints, 'new relative value')
plot_mean_trace_for_condition(all_block_changes, timepoints, 'change in relative value')

(28,)
(12,)
(12,)
(10,)
(14,)
(8,)
(10,)
(10,)
(14,)
(6,)
(8,)
(12,)
(12,)
(5,)
(6,)
(8,)
(10,)
(6,)
(5,)
(5,)
(7,)


In [29]:
fig, axs = plt.subplots(nrows=2,ncols=2)
sns.pointplot(x='new reward amounts', y='peak size', data=all_block_changes, ax=axs[0,0], ci='sd')
sns.pointplot(x='change in reward amounts', y='peak size', data=block_change_info, ax=axs[0,1], ci='sd')
sns.pointplot(x='new relative value', y='peak size', data=block_change_info, ax=axs[1,0], ci='sd')
sns.pointplot(x='change in relative value', y='peak size', data=block_change_info, ax=axs[1,1], ci='sd')

In [65]:
colours = cm.inferno(np.linspace(0,1, 5))
fig2, ax = plt.subplots(1,3)
for trial_type in range(1,6):
    try:
        one_side_data = all_reward_block_data[trial_type-1].contra_data
        x_vals = np.ones([len(one_side_data.trial_peaks)])*trial_type
        ax[0].hist(one_side_data.trial_peaks, color=colours[trial_type-1], alpha=0.4)
        ax[1].scatter(trial_type, np.mean(one_side_data.trial_peaks), color=colours[trial_type-1])
        ax[1].set_ylabel('Mean peak')
        ax[2].scatter(trial_type, np.median(one_side_data.trial_peaks), color=colours[trial_type-1])
        ax[2].set_ylabel('Median peak')
        print(np.mean(one_side_data.trial_peaks), np.median(one_side_data.trial_peaks))
    except:
        pass

0.8719974503007026 0.8838530247845447
0.734418515406605 0.5818760037034908
1.3165566302983154 1.1614116229862628
0.7800667694836998 0.663554277871089
0.8304953025800229 0.5631001717818181


In [217]:
np.median(np.asarray(all_trial_type_data[-2].contra_data.trial_peaks)/4)

0.15315428214827714

In [ ]:
np.median(np.asarray(all_trial_type_data[-2].contra_data.trial_peaks)/4)

In [209]:
np.asarray(one_side_data.trial_peaks)/4

array([0.14386391, 0.36703822, 0.03294286, 0.67309225, 0.1799292 ,
       0.26867563, 0.06190406])

In [199]:
np.median([0.13636681, 0.46637622, 0.51340394])

0.46637622

In [207]:
np.median(np.concatenate([np.asarray(one_side_data.trial_peaks),np.asarray([0.13636681, 0.46637622, 0.51340394])]))

0.5444297804051131